In [1]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

아래 실행 커맨드는 gpt_ckpt 폴더가 있지 않은 경우에만 실행.

In [1]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

100% [......................................................................] 460908853 / 460908853

In [2]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [3]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [5]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [6]:
generate_sent('이때', gpt_model, greedy=True)

'이때문에 일부 전문가들은 “이번 사건은 ‘제2의 삼성’을 꿈꾸는 삼성의 내부 사정을 잘 보여주는 사례”라고 평가했다.'

In [7]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때까지 보스턴 마라톤 대회에서 가장 많이 트랙을 돌았으며 그 뒤로 2009년 뉴욕 베이컨브리지 마라톤, 2010년 보스턴마라톤을 모두 석권했다.'

In [8]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]

In [32]:
sents = [s[:-1] for s in open('./test.txt', encoding='utf-8').readlines()]

In [35]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [36]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [37]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [38]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/10
19/19 [==============================] - 50s 3s/step - loss: 2.4224 - accuracy_function: 0.0613 - val_loss: 2.5528 - val_accuracy_function: 0.0771
Epoch 2/10
19/19 [==============================] - 47s 2s/step - loss: 1.8608 - accuracy_function: 0.0858 - val_loss: 2.4150 - val_accuracy_function: 0.0935
Epoch 3/10
19/19 [==============================] - 47s 2s/step - loss: 1.6730 - accuracy_function: 0.0994 - val_loss: 2.3661 - val_accuracy_function: 0.1040
Epoch 4/10
19/19 [==============================] - 48s 3s/step - loss: 1.5147 - accuracy_function: 0.1085 - val_loss: 2.3520 - val_accuracy_function: 0.1129
Epoch 5/10
19/19 [==============================] - 48s 3s/step - loss: 1.3625 - accuracy_function: 0.1179 - val_loss: 2.3788 - val_accuracy_function: 0.1213
Epoch 6/10
19/19 [==============================] - 49s 3s/step - loss: 1.2421 - accuracy_function: 0.1259 - val_loss: 2.4457 - val_accuracy_function: 0.1287
Epoch 7/10
19/19 [==============================] - 

In [39]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out\tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [40]:
generate_sent('이때', gpt_model, greedy=True)

'이때문에 족발보쌈은 족발에 굴이 들어가있어서 신선했어요~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

In [76]:
generate_sent('족발', gpt_model, top_k=0, top_p=0.95)

'족발 주문했는데 양이 줄어 아쉽네요 담에 또 주문할게요 굿나잇'

In [75]:
generate_sent('보쌈', gpt_model, top_k=0, top_p=0.95)

'보쌈을 먹을 수 있는 날이 며칠이나 될까요?...보쌈 먹고 싶지가 않아요. 보쌈을 시키지 않아도 돼요 다른 보쌈을 시켜먹을 수도 있어요 보쌈은 이렇게 족발보다는 족발이 더 맛있어요 보쌈'

In [53]:
generate_sent('막국수', gpt_model, top_k=0, top_p=0.95)

'막국수 1작은국수도 양념이 좋아요 막국수는 기름기가 적고 찰기가 있어서 먹는데 별로였어요'

In [54]:
generate_sent('김치', gpt_model, top_k=0, top_p=0.95)

'김치맛은 좋았는데 배달이 너무 오래 걸림 요 근래 보기드문 식성이라 다시 먹을려고 하는데....'

In [60]:
generate_sent('주문', gpt_model, top_k=0, top_p=0.95)

'주문한지 한 시간이나 지났는데 배송이 너무 늦은거 빼면 다좋은데요!!'

In [56]:
generate_sent('양도', gpt_model, top_k=0, top_p=0.95)

'양도 많고 맛있네요.. 음식배달하시는 분도 정말 친절하시구요.. 맛있게 먹었습니다!! 담에 또 먹겠습니다!!!'

In [62]:
generate_sent('가격', gpt_model, top_k=0, top_p=0.95)

'가격 대비 활용도가 높은 수고가이맛집 선별하여 드리고 싶습니다.배달해주시는 분은 친절하고 무조건 만족한다고 말씀해주시는 분이다. '

In [91]:
generate_sent('짜빠구리', gpt_model, top_k=0, top_p=0.95)

'짜빠구리파전은 톡 쏘는 식감이 특이하고 상추쌈도 막국수가 맛잇네요^^'